In [ ]:
sys.path.append('../')
import dask
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

import sqlalchemy
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from RtdRay import RtdRay
from mpl_toolkits.basemap import Basemap

from config import db_database, db_password, db_server, db_username

In [ ]:
def train_types_pie_chart(rtd_df):
    train_types = rtd_df['train_type']
    per_train_type = train_types.value_counts()
    per_train_type.plot.pie(figsize=(30, 30))

In [ ]:
def data_over_time(rtd_df):
    rtd_df['arr'].plot()

In [ ]:
def trains_per_station_map_bubble_plot(rtd_df):
    trains_per_station = rtd_df['station'].value_counts()


In [ ]:
def delay_over_time(delays):
    delays['arr_delay'] = delays['arr_changed_time'] - delays['arr']
    delays['arr_unix'] = delays['arr'] / 60
    delays['arr'] = dd.to_datetime(delays['arr'] - delays['arr'] % (60 * 60 * 24), unit='s')
    delays = delays.groupby('arr').agg({'arr_unix':['std'], 'arr_delay': ['mean', 'std', 'max', 'min']})
    with ProgressBar():
        return delays.compute()

In [ ]:
rtd = RtdRay(notebook=True)
delays = rtd.load_data(columns=['arr', 'arr_changed_time'])
grouped_by = delay_over_time(delays)

In [ ]:
plot = grouped_by.reset_index().plot.scatter(y=('arr_delay', 'max'), x=('arr'))

In [ ]:
grouped_by.reset_index()

In [ ]:
rtd.load_data(columns=['arr', 'arr_changed_time'])['arr_changed_time'].head()

In [ ]:
delays = rtd.load_data(columns=['arr', 'arr_changed_time', 'arr_changed_platform', 'arr_changed_status', 'stay_time'])
delays_df = delays.compute()
delays_df['arr_changed_time'] - delays_df['arr']